# Google Maps geocoder

In [1]:
import pandas as pd
import requests
import logging
import time
from creds import *

In [2]:
#request cache
key = my_api_key
API_KEY = my_api_key

In [3]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

In [4]:
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

In [5]:
BACKOFF_TIME = 1
output_filename = 'data/output/mass_shootings_involving_deaths_addresses.csv'
input_filename = '../gun-violence/data/processed/mass_shootings_involving_deaths.csv'
address_column_name = 'full_address'
RETURN_FULL_RESULTS = True
data = pd.read_csv(input_filename, encoding='utf8')

In [6]:
if address_column_name not in data.columns:
    raise ValueError("Missing Address column in input data")

addresses = data[address_column_name].tolist()

def get_google_results(full_address, api_key=my_api_key, return_full_response=True):
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + '&key={}'.format(api_key)
        
    results = requests.get(geocode_url)
    results = results.json()
    
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            "google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
         
    output['input_string'] = full_address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

results = []
for address in addresses:
    geocoded = False
    while geocoded is not True:
        try:
            geocode_result = get_google_results(address, my_api_key, return_full_response=RETURN_FULL_RESULTS)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(address))
            logger.error("Skipping!")
            geocoded = True
            
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            logger.info("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            if geocode_result['status'] != 'OK':
                logger.warning("Error geocoding {}: {}".format(address, geocode_result['status']))
            logger.debug("Geocoded: {}: {}".format(address, geocode_result['status']))
            results.append(geocode_result)           
            geocoded = True

    if len(results) % 500 == 0:
        logger.info("Completed {} of {} address".format(len(results), len(addresses)))
            
    if len(results) % 500 == 0:
        pd.DataFrame(results).to_csv("{}_bak".format(output_filename))

logger.info("Finished geocoding all addresses")
pd.DataFrame(results).to_csv(output_filename, encoding='utf8')

Geocoded: 6161 S Yale Ave, Tulsa, OK: OK
Geocoded: 999 Pipestone St, Benton Harbor, MI: OK
Geocoded: N Delaware Ave and E Tioga St, Philadelphia, PA: OK
Geocoded: N 58th St and W Windsor Ave, Phoenix, AZ: OK
Geocoded: 104 Elm St, Taft, OK: OK
Geocoded: Cowden Ave, Merced, CA: OK
Geocoded: 4400 W Walton St, Chicago, IL: OK
Geocoded: 3800 E Pikes Pike Ave, Colorado Springs, CO: OK
Geocoded: 2233 N 1st St, Fresno, CA: OK
Geocoded: 10711 185th Ave, Stanwood, MI: OK
Geocoded: 715 Old Carrizo Rd, Uvalde, TX: OK
Geocoded: 715 Old Carrizo Rd, Uvalde, TX: OK
Geocoded: 1204 Rosemare Ct, Goshen, IN: OK
Geocoded: 2331 Old Dixie Hwy, Kissimmee, FL: OK
Geocoded: 3606 Highland Ave, Highland, CA: OK
Geocoded: St. Claude Ave and Forstall St, New Orleans, LA: OK
Geocoded: 800 N State St, Chicago, IL: OK
Geocoded: 2500 Fordham St, Palo Alto (East Palo Alto), CA: OK
Geocoded: 24301 El Toro Rd, Laguna Woods, CA: OK
Geocoded: 1003 N Fillmore St, Amarillo, TX: OK
Geocoded: 8729 Airline Dr, Houston, TX: OK
Ge